In [ ]:
pip install  google-cloud-bigquery

In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
pip install google-cloud-bigquery-storage

In [37]:
PROJECT='sqlpandas2020'
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import os 
credentials = service_account.Credentials.from_service_account_file('/Users/yonnyclinton/Downloads/sqlpandas2020-0eb79fba01d1.json') 
bq = bigquery.Client(credentials=credentials,project=PROJECT)


In [2]:
query = """
SELECT
a.*
,condition_concept_id
,concept_name
,CASE WHEN(b.death_date IS NOT NULL) THEN 1 ELSE 0 END  AS DEATH_INDX
,EXTRACT(YEAR FROM (CURRENT_DATE()) )- year_of_birth AS AGE
FROM `sqlpandas2020.SQLPANDASEST.person`  a
LEFT JOIN `sqlpandas2020.SQLPANDASEST.DEATH` b ON a.person_id=b.person_id
LEFT JOIN (SELECT
            person_id
            ,condition_concept_id
            , concept_name
            FROM (SELECT person_id ,condition_concept_id
                          ,ROW_NUMBER() OVER(PARTITION BY person_id ORDER BY condition_era_start_date DESC) AS ROW
                      FROM (SELECT DISTINCT
                              person_id, condition_concept_id, condition_era_start_date
                              FROM  `sqlpandas2020.SQLPANDASEST.condition_era`)) A
            LEFT JOIN `sqlpandas2020.SQLPANDASEST.CONCEPT` B ON B.concept_id=A.condition_concept_id
            WHERE ROW =1 ) C ON  a.person_id=C.person_id 
LIMIT 100000

"""

In [3]:
df = bq.query(query).to_dataframe()
print(df.describe())

       race_concept_id  ethnicity_concept_id    location_id  provider_id  \
count    100000.000000          1.000000e+05  100000.000000          0.0   
mean       8002.363100          3.800356e+07     692.963200          NaN   
std        2046.750411          1.475287e-01     706.008685          NaN   
min           0.000000          3.800356e+07       1.000000          NaN   
25%        8527.000000          3.800356e+07     143.000000          NaN   
50%        8527.000000          3.800356e+07     436.000000          NaN   
75%        8527.000000          3.800356e+07    1031.000000          NaN   
max        8527.000000          3.800356e+07    3182.000000          NaN   

       care_site_id  gender_source_concept_id  race_source_concept_id  \
count           0.0                       0.0                     0.0   
mean            NaN                       NaN                     NaN   
std             NaN                       NaN                     NaN   
min             NaN    

In [12]:
x = pd.DataFrame(pd.read_csv("/Users/yonnyclinton/Documents/ejercicio-28-nov.csv") )

In [11]:
path = os.getcwd()
print ("The current working directory is %s" % path)

The current working directory is /Users/yonnyclinton


In [13]:
x.head()

,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,...,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,condition_concept_id,concept_name,DEATH_INDX,AGE
0,8527,38003564,2007,NaN,NaN,4D6CBEB427A7A96C,1,NaN,1,NaN,...,849432,8507,1958,1,1,NaN,444445.0,Benign neoplasm of skin of lower limb,0,62
1,8527,38003564,1002,NaN,NaN,38FC7AC2E10A9AFE,1,NaN,1,NaN,...,2119843,8507,1957,1,1,NaN,433577.0,Hammer toe,0,63
2,8527,38003564,924,NaN,NaN,599DB431C3816D0B,1,NaN,1,NaN,...,157121,8507,1955,1,1,NaN,NaN,NaN,0,65
3,8527,38003564,45,NaN,NaN,4B64660B53D74FCF,1,NaN,1,NaN,...,1662633,8507,1910,1,1,NaN,200219.0,Abdominal pain,0,110
4,8527,38003564,361,NaN,NaN,6BF81D361281049D,1,NaN,1,NaN,...,630852,8507,1977,1,1,NaN,195306.0,Gastroduodenitis,0,43


In [14]:
df['AGE'].min()

37

In [15]:
df['AGE'].max()

111

In [16]:
df['year_of_birth'].min()

1909

In [17]:
df['year_of_birth'].max()

1983

In [23]:
mean=df['AGE'].mean()

In [19]:
df['AGE'].quantile([0.2 , 0.5 ,0.7 , 0.9 ])

0.2    77.0
0.5    84.0
0.7    90.0
0.9    98.0
Name: AGE, dtype: float64

In [24]:
standart=df['AGE'].std()

In [25]:
li= mean - standart

In [26]:
ls = mean + standart

In [29]:
df[df.AGE < li]

,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,...,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,condition_concept_id,concept_name,DEATH_INDX,AGE
1,8527,38003564,1017,NaN,NaN,626326E496AB39E2,1,NaN,1,NaN,...,1673053,8507,1956,1,1,NaT,81902.0,Urinary tract infectious disease,0,64
15,8527,38003564,20,NaN,NaN,9DF611E1BF00F9E0,2,NaN,1,NaN,...,1118975,8532,1967,1,1,NaT,133810.0,Hyperparathyroidism due to renal insufficiency,0,53
21,8527,38003564,208,NaN,NaN,DA9562CB563064BA,2,NaN,1,NaN,...,1262507,8532,1953,1,1,NaT,374888.0,Dementia associated with another disease,0,67
23,8527,38003564,620,NaN,NaN,A1A688AA7FCE4488,2,NaN,1,NaN,...,73534,8532,1956,1,1,NaT,195083.0,Left lower quadrant pain,0,64
42,8527,38003564,1362,NaN,NaN,FAB1833D59E94F69,1,NaN,1,NaN,...,1509938,8507,1950,1,1,NaT,133468.0,Disorder of extremity,0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99943,8527,38003564,89,NaN,NaN,0BE3AD306C6F3E93,1,NaN,1,NaN,...,1168774,8507,1955,5,1,NaT,NaN,None,0,65
99981,8527,38003564,1980,NaN,NaN,9417DEFDA27158EA,1,NaN,1,NaN,...,1579463,8507,1956,5,1,NaT,NaN,None,0,64
99989,8527,38003564,9,NaN,NaN,D5095FA07F31C2BF,1,NaN,1,NaN,...,678465,8507,1960,5,1,NaT,198194.0,Female genital organ symptoms,0,60
99996,8527,38003564,18,NaN,NaN,3776E509F2DE8408,2,NaN,1,NaN,...,2119129,8532,1951,5,1,NaT,141932.0,Senile hyperkeratosis,1,69


In [30]:
df[df.AGE > ls]

,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,...,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,condition_concept_id,concept_name,DEATH_INDX,AGE
10,8527,38003564,85,NaN,NaN,7E07A47CE2850CF6,2,NaN,1,NaN,...,1569359,8532,1913,1,1,NaT,80816.0,Degeneration of intervertebral disc,0,107
14,8527,38003564,249,NaN,NaN,B9EC482275095892,2,NaN,1,NaN,...,2294865,8532,1923,1,1,NaT,433774.0,Orbital myositis,0,97
18,8527,38003564,37,NaN,NaN,127920A137A975DB,2,NaN,1,NaN,...,2102543,8532,1919,1,1,NaT,436659.0,Iron deficiency anemia,0,101
20,8527,38003564,901,NaN,NaN,0B8108A597EF23D0,2,NaN,1,NaN,...,2215724,8532,1919,1,1,NaT,NaN,None,0,101
31,8527,38003564,57,NaN,NaN,F5A62344F0FDFEE5,1,NaN,1,NaN,...,1391449,8507,1920,1,1,NaT,443612.0,Chronic kidney disease stage 4,0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99980,8527,38003564,18,NaN,NaN,810B0AC18E0364F3,2,NaN,1,NaN,...,175044,8532,1921,5,1,NaT,77670.0,Chest pain,0,99
99986,8527,38003564,9,NaN,NaN,5DB66D4083147396,1,NaN,1,NaN,...,2136693,8507,1921,5,1,NaT,437827.0,Pure hypercholesterolemia,1,99
99987,8527,38003564,260,NaN,NaN,2BDCD7B211011F44,1,NaN,1,NaN,...,950549,8507,1913,5,1,NaT,376691.0,Old partial retinal detachment,0,107
99992,8527,38003564,736,NaN,NaN,37BA03544AC9E32D,1,NaN,1,NaN,...,1770427,8507,1914,5,1,NaT,81454.0,Injury of finger,0,106


In [32]:
df.groupby(["AGE"]).count()

,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,...,ethnicity_source_concept_id,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,condition_concept_id,concept_name,DEATH_INDX
AGE,,,,,,,,,,,,,,,,,,,,,
37,154,154,154,0,0,154,154,0,154,0,...,0,154,154,154,154,154,0,130,130,154
38,122,122,122,0,0,122,122,0,122,0,...,0,122,122,122,122,122,0,103,103,122
39,135,135,135,0,0,135,135,0,135,0,...,0,135,135,135,135,135,0,113,113,135
40,149,149,149,0,0,149,149,0,149,0,...,0,149,149,149,149,149,0,123,123,149
41,146,146,146,0,0,146,146,0,146,0,...,0,146,146,146,146,146,0,122,122,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,505,505,505,0,0,505,505,0,505,0,...,0,505,505,505,505,505,0,446,446,505
108,465,465,465,0,0,465,465,0,465,0,...,0,465,465,465,465,465,0,387,387,465
109,505,505,505,0,0,505,505,0,505,0,...,0,505,505,505,505,505,0,420,420,505


In [33]:
#from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from datetime import datetime, timedelta
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import sklearn
import random
from sklearn.metrics import confusion_matrix
from inspect import getmembers
import pickle

In [35]:
#rellenar null values
training_data=df[~df['DEATH_INDX'].isnull()]
print (training_data.shape)

(100000, 22)


In [38]:
#Impute missing with zero and basic Data Cleaning
training_data=training_data.replace(np.inf, np.nan)
#validation=validation.replace(np.inf, np.nan)
#test_data=test_df.replace(np.inf, np.nan)

In [39]:
training_data.fillna(0,inplace=True)
#validation.fillna(0,inplace=True)
#test_data.fillna(0,inplace=True)

In [40]:
label=['DEATH_INDX']

In [41]:
features=[ 'ethnicity_concept_id',	'location_id','condition_concept_id',	'AGE']
features=[x for x in list(training_data.columns) if x in features]

In [42]:
#Gradient Boosting
md = GBDT(random_state=10) # No hyperparam tuning
P=md.fit(training_data[features],training_data[label]) # Fit the model

/Users/yonnyclinton/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
